<a href="https://colab.research.google.com/github/shouvikguha/Division-Diviner/blob/main/England/Premier_League/PLScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests ## helps make HTTP requests
from bs4 import BeautifulSoup ## helps with parsing HTML
import pandas as pd ## helps with analysing our tabular data
import time ## helps prevent site from blocking requests

In [ ]:
# pl_standings_links = "https://fbref.com/en/comps/9/Premier-League-Stats"
pl_standings_links = "https://fbref.com/en/comps/9/2023-2024/2023-2024-Premier-League-Stats"

## look into using above imports to get prev season if curr season not started

In [ ]:
pl_standings_data = requests.get(pl_standings_links) ## using requests to "get" data from the url

In [49]:
pl_data = BeautifulSoup(pl_standings_data.text)
# print(pl_data)
pl_table = pl_data.select("table.stats_table")[0] ## CSS selector which gets table elements with class name stats_table

IndexError: list index out of range

In [50]:
urls = pl_table.find_all("a") ## finds by HTML tags, in this case anchor tags
urls = [u.get("href") for u in urls if u.get("href") and "/squads" in u.get("href")] ## extract href prop from every anchor tag in list that contains string squads
## conditional set up this way to prevent TypeError in case comparison between string and None made
urls = [f"https://fbref.com{u}" for u in urls] ## complete the extracted urls and turn from relative into absolute url
teams = urls
teams

['https://fbref.com/en/squads/b8fd03ef/2023-2024/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/2023-2024/Arsenal-Stats',
 'https://fbref.com/en/squads/822bd0ba/2023-2024/Liverpool-Stats',
 'https://fbref.com/en/squads/8602292d/2023-2024/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/2023-2024/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/cff3d9bb/2023-2024/Chelsea-Stats',
 'https://fbref.com/en/squads/b2b47a98/2023-2024/Newcastle-United-Stats',
 'https://fbref.com/en/squads/19538871/2023-2024/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/2023-2024/West-Ham-United-Stats',
 'https://fbref.com/en/squads/47c64c55/2023-2024/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/d07537b9/2023-2024/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/4ba7cbea/2023-2024/Bournemouth-Stats',
 'https://fbref.com/en/squads/fd962109/2023-2024/Fulham-Stats',
 'https://fbref.com/en/squads/8cec06e1/2023-2024/Wolverhampton-Wanderers-Stat

In [51]:
team1 = teams[0]
team_data = requests.get(team1) ## gets data from the url of team 1

In [52]:
matches = pd.read_html(team_data.text, match = "Scores & Fixtures")[0]
## pass in our HTML and we want to only get the table tag w a positive match on "Scores & Fixtures" as a string inside it
## read_html scans tabular data and turns into list
matches.head() ## dataframe for matches played by team ready


<ipython-input-52-c2f34573cfcd>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(team_data.text, match = "Scores & Fixtures")[0]


ValueError: No tables found

In [ ]:
team_spec_data = BeautifulSoup(team_data.text)
shooting_url = team_spec_data.find_all("a")
shooting_url = [s.get("href") for s in shooting_url if s.get("href") and "all_comps/shooting" in s.get("href")] ## same process here to get team shooting (stats) links
shooting_url

[]

In [ ]:
shooting_data = requests.get(f"https://fbref.com{shooting_url[0]}")
shots = pd.read_html(shooting_data.text, match = "Shooting")[0]
shots.head()

IndexError: list index out of range

In [ ]:
shots.columns = shots.columns.droplevel() ## getting rid of top index level to prevent multi-level index ("For [team] ... standard, expected, etc"), helps clean things up and process easier
shots.head() ## shooting stats dataframe ready

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,13.9,0.0,0,0,1.9,1.9,0.12,1.1,1.1,Match Report
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,17.9,0.0,0,0,1.0,1.0,0.07,0.0,0.0,Match Report
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,17.3,2.0,0,1,3.5,2.8,0.10,-1.5,-0.8,Match Report


In [ ]:
team_dataset = matches.merge(shots[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on= "Date")
## merged to include stats incl. shots, shots on target, avg distance of shot, fk, pk, pk attempted
print(matches.shape)
print(team_dataset.shape)
print(shots.shape)

(57, 19)
(57, 25)
(58, 26)


In [ ]:
years = list(range(2023, 2018, -1)) ## adjust this to get last 5 based on new or continuing season
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
for year in years:
    standing_info = requests.get(standings_url)
    pl_standings = BeautifulSoup(standing_info.text)
    standings_table = pl_standings.select("table.stats_table")[0]

    urls = [u.get("href") for u in standings_table.find_all("a") if "/squads/" in u.get("href")]
    pl_squad_urls = [f"https://fbref.com{u}" for u in urls]

    prev_season = pl_standings.select("a.prev")[0].get("href")
    p_standings_url = f"https://fbref.com{prev_season}"
    print(p_standings_url)

    for u in pl_squad_urls:
        team = u.split("/")[-1].replace("-Stats", "").replace("-", " ") ## extract only team name and drop all other chars
        team_data = requests.get(u)
        matches = pd.read_html(team_data.text,match = "Scores & Fixtures")[0]
        team_info = BeautifulSoup(team_data.text)
        shooting_link_for_team = [t.get("href") for t in team_info.find_all("a") if t.get("href") and 'all_comps/shooting/' in t.get("href")]
        # shooting_link_for_team = [l.get("href") for l in team_info.find_all("a")]
        # shooting_link_for_team = [l for l in shooting_link_for_team if l and "all_comps/shooting/" in l]

        # print(shooting_stat_for_team[0])
        shooting_data = requests.get(f"https://fbref.com{shooting_link_for_team[0]}")
        shots_info = pd.read_html(shooting_data.text, match = "Shooting")[0]
        shots_info.columns = shots_info.columns.droplevel()
        try:
            team_dataset = matches.merge(shots_info[["Date","Sh","SoT","Dist","FK","PK","PKatt"]],on = "Date")
        except ValueError:
            continue
        team_dataset = team_dataset[team_dataset["Comp"] == "Premier League"]

        team_dataset["Season"] = year
        team_dataset["Team"] = team
        all_matches.append(team_dataset)
        time.sleep(20)

<Response [429]> here


IndexError: list index out of range